In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score,classification_report

 

We have the the dataset for this task in making_dataset.csv you can modify it as your needs

In [18]:
#We know that our data contains 'N/A' values for Dosage and frequency columns for healthy patients, 
#I am creating model that can predict those values for healthy patients so I will keep N/A as it is

df=pd.read_csv(r'Respiratory_Sound_Database\medicine_prescription.csv',keep_default_na=False)
df.head()

,Patient ID,Age,Gender,Smoking Status,Disease,Medication Prescribed,Dosage,Frequencey
0,1,83,Female,Active-smoker,Bronchiolitis,Drug D,50mg,Twice daily
1,2,41,Male,Non-smoker,Bronchiectasis,Drug C,50mg,Once daily
2,3,35,Male,Ex-smoker,Pneumonia,Drug C,5mg,Three times daily
3,4,13,Male,Active-smoker,LRTI,Drug C,20mg,Once daily
4,5,70,Male,Active-smoker,URTI,Drug C,5mg,Three times daily


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Patient ID             3000 non-null   int64 
 1   Age                    3000 non-null   int64 
 2   Gender                 3000 non-null   object
 3   Smoking Status         3000 non-null   object
 4   Disease                3000 non-null   object
 5   Medication Prescribed  3000 non-null   object
 6   Dosage                 3000 non-null   object
 7   Frequencey             3000 non-null   object
dtypes: int64(2), object(6)
memory usage: 187.6+ KB
